## Ансамбли и полносвязные нейронные сети
В этом ноутбуке вам нужно обучить модели на датасете классификации из предыдущего ноутбука и сравнить результаты. Вам будет предоставлен baseline, на основе которого вы будете доделывать предсказывающие модели. Оценка лабы будет зависеть от ROC-AUC на тестовых данных по следующим критериям:
\
AUC - на тестовых данных
- $AUC \leq 0.76$ - 0 баллов
- $0.76 < AUC \leq 0.77$ - 2 балла
- $0.77 < AUC \leq 0.78$ - 4 балла
- $0.78 < AUC \leq 0.79$ - 6 баллов
- $0.79 < AUC \leq 0.80$ - 8 баллов
- $AUC > 0.80$ - 10 баллов


In [ ]:
# импорт необходимых моделей
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

# импорт метриков для оценки производительности
from sklearn.metrics import roc_auc_score, precision_score, recall_score, roc_curve, accuracy_score

# импорт функции для разделения данных
from sklearn.model_selection import train_test_split

# импорт библиотек визуализации 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
# загрузка CSV, добавление разделителя
data = pd.read_csv('german.csv', sep=';')
print(data.head())

# извлечение всех столбцов кроме первого
X = data.iloc[:, 1:].to_numpy()
y = data.iloc[:, 0].to_numpy()

# разделение данных на обучение и тестировку
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
plt.hist(y_train, bins=2, edgecolor='k')
plt.xticks([0, 1])
plt.xlabel('Class (0: Non-Creditworthy, 1: Creditworthy)')
plt.ylabel('Count')
plt.title('Distribution of Classes in Training Data')
plt.show()

In [ ]:
# Обучение Random Forest
# установка параметров для модели
rf_model = RandomForestClassifier(
    n_estimators=26, # количество деревьев
    max_depth=5, # максимальная глубина
    min_samples_split=50, # минимальное количество образцов разделения 
    min_samples_leaf=5, # мин. кол-во образцов в листе
    max_features=1.0, # макс. кол-во признаков для поиска лучшего разделения 
    max_leaf_nodes=55, # макс. кол-во листовых узлов 
    random_state=42, # устан. значения 
    max_samples=269, # макс. кол-во образцов для обучения
    criterion="entropy", # критерий оценки качества разделения
    class_weight='balanced_subsample') 
rf_model.fit(X_train, y_train)

# Прогноз на тестовых данных
rf_pred = rf_model.predict(X_test)
rf_pred_auc = rf_model.predict_proba(X_test)[:, 1] # вероятность для полож. класса

# Расчет метрик для Random Forest
rf_roc_auc = roc_auc_score(y_test, rf_pred_auc)
rf_accuracy = accuracy_score(y_test, rf_pred)
rf_precision = precision_score(y_test, rf_pred)
rf_recall = recall_score(y_test, rf_pred)

# вывод метриков производительности
print("Random Forest метрики:")
print(f"ROC AUC: {rf_roc_auc:.2f}")
print(f"Accuracy: {rf_accuracy:.2f}")
print(f"Precision: {rf_precision:.2f}")
print(f"Recall: {rf_recall:.2f}")

# Обучение Gradient Boosting
gb_model = GradientBoostingClassifier(
    n_estimators=167, # кол-во деревьев
    random_state=42, # установка значения seed
    learning_rate = 0.099, # скорость обучения
    min_samples_split=167, # мин. кол-во образцов для разделения узла 
    min_samples_leaf=71, # мин. кол-во образцов в листе
    max_features=9, # макс. кол-во признаков для поиска лучшего разделения
    max_depth=2) # макс. глубина дерева
gb_model.fit(X_train, y_train)

# Прогноз на тестовых данных
gb_pred = gb_model.predict(X_test)
gb_pred_auc = gb_model.predict_proba(X_test)[:, 1]

# Расчет метрик для Gradient Boosting
gb_roc_auc = roc_auc_score(y_test, gb_pred_auc)
gb_accuracy = accuracy_score(y_test, gb_pred)
gb_precision = precision_score(y_test, gb_pred)
gb_recall = recall_score(y_test, gb_pred)

print("\nGradient Boosting метрики:")
print(f"ROC AUC: {gb_roc_auc:.2f}")
print(f"Accuracy: {gb_accuracy:.2f}")
print(f"Precision: {gb_precision:.2f}")
print(f"Recall: {gb_recall:.2f}")

In [ ]:
# Обучение MLP (Multi-Layer Perceptron) нейронной сети
mlp_model = MLPClassifier(
    hidden_layer_sizes=(30,), #размер скрытого слоя
    max_iter=500, # максимальное количество итераций
    random_state=42, # установка для воспроизводимости результатов
    activation='logistic', # функция активации
    n_iter_no_change=23) # итерации без изменения, после которого остановка обучения

mlp_model.fit(X_train, y_train)

# Прогноз на тестовых данных
mlp_pred = mlp_model.predict(X_test)
mlp_pred_log_proba = mlp_model.predict_log_proba(X_test)[:,1]

# Расчет метрик для MLP нейронной сети
mlp_roc_auc = roc_auc_score(y_test, mlp_pred_log_proba)
mlp_accuracy = accuracy_score(y_test, mlp_pred)
mlp_precision = precision_score(y_test, mlp_pred)
mlp_recall = recall_score(y_test, mlp_pred)

# вывод производительности модели 
print("\nMLP (Neural Network) метрики:")
print(f"ROC AUC: {mlp_roc_auc:.3f}")
print(f"Accuracy: {mlp_accuracy:.2f}")
print(f"Precision: {mlp_precision:.2f}")
print(f"Recall: {mlp_recall:.2f}")

## Экспериментируйте
Для получения лучшего качества придется поэкспериментировать. Подсказка: попробуйте оптимизировать гиперпараметры модели